In [1]:


import glob
import numpy as np
import pandas as pd 



In [2]:
path='./drive/MyDrive/head_ct/head_ct'
glob.glob(path+'/*')

['./drive/MyDrive/head_ct/head_ct/003.png',
 './drive/MyDrive/head_ct/head_ct/010.png',
 './drive/MyDrive/head_ct/head_ct/000.png',
 './drive/MyDrive/head_ct/head_ct/005.png',
 './drive/MyDrive/head_ct/head_ct/002.png',
 './drive/MyDrive/head_ct/head_ct/007.png',
 './drive/MyDrive/head_ct/head_ct/008.png',
 './drive/MyDrive/head_ct/head_ct/011.png',
 './drive/MyDrive/head_ct/head_ct/004.png',
 './drive/MyDrive/head_ct/head_ct/012.png',
 './drive/MyDrive/head_ct/head_ct/006.png',
 './drive/MyDrive/head_ct/head_ct/009.png',
 './drive/MyDrive/head_ct/head_ct/001.png',
 './drive/MyDrive/head_ct/head_ct/104.png',
 './drive/MyDrive/head_ct/head_ct/105.png',
 './drive/MyDrive/head_ct/head_ct/066.png',
 './drive/MyDrive/head_ct/head_ct/077.png',
 './drive/MyDrive/head_ct/head_ct/093.png',
 './drive/MyDrive/head_ct/head_ct/090.png',
 './drive/MyDrive/head_ct/head_ct/075.png',
 './drive/MyDrive/head_ct/head_ct/089.png',
 './drive/MyDrive/head_ct/head_ct/083.png',
 './drive/MyDrive/head_ct/head_c

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
def load_samples_as_images(pathX, pathY,img_width, img_height):
    files = sorted(glob.glob(pathX))
    labels_df = pd.read_csv(pathY)
    Y = np.array(labels_df[' hemorrhage'].tolist())
    X=[]

    for i, _file in enumerate(files):
        img=load_img(_file, target_size=(150,150))
        img=img_to_array(img)
        img=img/255.0
        X.append(img)
    X=np.array(X)
    return X, Y

In [15]:
import cv2
pathX = "./drive/MyDrive/head_ct/head_ct/*.png"
pathY = './drive/MyDrive/labels.csv'
img_width=150
img_height=150
images, Y = load_samples_as_images(pathX, pathY, img_width, img_height)

In [16]:
from sklearn.model_selection import train_test_split
# split the dataset into train (80%), validation (10%) and test (10%) sets.
train_images, test_images, train_labels, test_labels = train_test_split(images, Y, test_size=0.2,
                                                                            random_state=1)

In [36]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
#step1
# file_resnet='/kaggle/input/vgg16/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model=ResNet50( input_shape=(150,150,3),
                                  include_top=False,
                                  weights='imagenet'
                                   )
#step2
for layer in pretrained_model.layers:
     layer.trainable = False

# pretrained_model.summary()
        
#step3        
last_layer = pretrained_model.get_layer('conv5_block3_out')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output = last_layer.output

#step4
x = layers.Flatten()(last_output)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.4)(x)                  
x = layers.Dense(1, activation='sigmoid')(x)


#step5
model_resnet = Model(pretrained_model.input, x) 

#step6
model_resnet.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])


last layer of vgg : output shape:  (None, 5, 5, 2048)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [37]:
model_resnet.fit(train_images,train_labels,epochs=100,validation_data=(test_images,test_labels))

Epoch 1/100
5/5 [==============================] - 17s 3s/step - loss: 49.0101 - acc: 0.5813 - val_loss: 24.8813 - val_acc: 0.4250
Epoch 2/100
3/5 [=================>............] - ETA: 4s - loss: 23.7050 - acc: 0.5625

KeyboardInterrupt: ignored